# Element Class

Exploring ways to turn elements from string names into objects that can be added/multiplied.

In [1]:
from finite_algebras import *

In [84]:
class Element():
    
    def __init__(self, name, algebra):
        
        self.__can_add = False
        self.__can_subtract = False
        self.__can_multiply = False
        self.__can_divide = False
        
        if isinstance(algebra, FiniteAlgebra):
            self.__algebra = algebra
            self.__can_add = True
            self.__can_subtract = self.__algebra.has_inverses()
            if isinstance(algebra, Ring):
                self.__can_multiply = True
                if isinstance(algebra, Field):
                    self.__can_divide = True
        else:
            raise ValueError(f"algebra must be a FiniteAlgebra")
            
        if isinstance(name, str):
            if name in self.__algebra:
                self.__name = name
            else:
                raise ValueError(f"name must be an element of algebra")
        else:
            raise ValueError(f"name must be a string")

    @property
    def name(self):
        return self.__name
    
    @property
    def algebra(self):
        return self.__algebra
    
    @property
    def can_add(self):
        return True
    
    @property
    def can_subtract(self):
        return self.__can_subtract
    
    @property
    def can_multiply(self):
        return self.__can_multiply
    
    @property
    def can_divide(self):
        return self.__can_divide
    
    def __str__(self):
        return self.__name
    
    def __repr__(self):
        return repr(self.__name)
    
    def __add__(self, other):
        elem = self.__algebra.op(self.__name, other.name)
        return Element(elem, self.__algebra)
    
    def __sub__(self, other):
        if self.__can_subtract:
            elem = self.__algebra.sub(self.__name, other.name)
            return Element(elem, self.__algebra)
        else:
            raise ValueError(f"{self.__algebra.name} does not support subtraction")
    
    def __neg__(self):
        if self.__can_subtract:
            elem = self.__algebra.inv(self.__name)
            return Element(elem, self.__algebra)
            
    def __mul__(self, other):
        if self.__can_multiply:
            elem = self.__algebra.mult(self.__name, other.name)
            return Element(elem, self.__algebra)
        else:
            raise ValueError(f"{self.__algebra.name} does not support multiplication")
    
    def __truediv__(self, other):
        if self.__can_divide:
            elem = self.__algebra.div(self.__name, other.name)
            return Element(elem, self.__algebra)
        else:
            raise ValueError(f"{self.__algebra.name} does not support division")
    
    def __pow__(self, n):
        result = self
        if self.__can_multiply:
            if isinstance(n, int) and n > 0:
                for _ in range(n - 1):
                    result = result * self
            else:
                raise ValueError(f"n = {n}, but the power must be a positive integer.")
        else:
            raise ValueError(f"{self.__algebra.name} does not support multiplication")
        return result
    
    def __key(self):
        return tuple([self.__name, self.__algebra.__hash__()])

    def __hash__(self):
        return hash(self.__key)

    def __eq__(self, other):
        if isinstance(other, Element):
            return self.__key() == other.__key() and self.__algebra == other.algebra
        else:
            return NotImplemented

In [137]:
def element_map(algebra):
    """Returns a dictionary where element names (str) are keys and the corresponding Elements
    are the values."""
    return {elem : Element(elem, algebra) for elem in algebra.elements}

## Test Group

In [85]:
s3 = generate_symmetric_group(3)
s3.about(use_table_names=True)


** Group **
Name: S3
Instance ID: 4483941456
Description: Autogenerated symmetric Group on 3 elements
Order: 6
Identity: '(1, 2, 3)'
Commutative? No
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0 '(1, 2, 3)' '(1, 2, 3)'       1
      1 '(1, 3, 2)' '(1, 3, 2)'       2
      2 '(2, 1, 3)' '(2, 1, 3)'       2
      3 '(2, 3, 1)' '(3, 1, 2)'       3
      4 '(3, 1, 2)' '(2, 3, 1)'       3
      5 '(3, 2, 1)' '(3, 2, 1)'       2
Cayley Table (showing names):
[['(1, 2, 3)', '(1, 3, 2)', '(2, 1, 3)', '(2, 3, 1)', '(3, 1, 2)', '(3, 2, 1)'],
 ['(1, 3, 2)', '(1, 2, 3)', '(3, 1, 2)', '(3, 2, 1)', '(2, 1, 3)', '(2, 3, 1)'],
 ['(2, 1, 3)', '(2, 3, 1)', '(1, 2, 3)', '(1, 3, 2)', '(3, 2, 1)', '(3, 1, 2)'],
 ['(2, 3, 1)', '(2, 1, 3)', '(3, 2, 1)', '(3, 1, 2)', '(1, 2, 3)', '(1, 3, 2)'],
 ['(3, 1, 2)', '(3, 2, 1)', '(1, 3, 2)', '(1, 2, 3)', '(2, 3, 1)', '(2, 1, 3)'],
 ['(3, 2, 1)', '(3, 1, 2)', '(2, 3, 1)', '(2, 1, 3)', '(1, 3, 2)', '(1, 2, 3)']]


In [89]:
a1 = Element('(2, 1, 3)', s3)
a1

'(2, 1, 3)'

In [90]:
elem = a1
print(elem.can_add)
print(elem.can_subtract)
print(elem.can_multiply)
print(elem.can_divide)

True
True
False
False


In [91]:
a2 = Element('(3, 2, 1)', s3)
a2

'(3, 2, 1)'

In [92]:
a1 + a2

'(3, 1, 2)'

In [94]:
(a2 - a1) + a1 == a2

True

In [95]:
try:
    print(a1 * a2)
except Exception as exc:
    print(exc)

S3 does not support multiplication


In [96]:
try:
    print(a1 / a2)
except Exception as exc:
    print(exc)

S3 does not support division


In [97]:
a3 = Element('(2, 3, 1)', s3)
a3

'(2, 3, 1)'

In [98]:
-a3

'(3, 1, 2)'

In [99]:
try:
    print(a1**2)
except Exception as exc:
    print(exc)

S3 does not support multiplication


## Test Monoid

In [100]:
m5 = generate_commutative_monoid(5)
m5.about(use_table_names=True)


** Monoid **
Name: M5
Instance ID: 5070292048
Description: Autogenerated commutative Monoid of order 5
Order: 5
Identity: a1
Associative? Yes
Commutative? Yes
Cyclic?: No
Elements: ['a0', 'a1', 'a2', 'a3', 'a4']
Has Inverses? No
Cayley Table (showing names):
[['a0', 'a0', 'a0', 'a0', 'a0'],
 ['a0', 'a1', 'a2', 'a3', 'a4'],
 ['a0', 'a2', 'a4', 'a1', 'a3'],
 ['a0', 'a3', 'a1', 'a4', 'a2'],
 ['a0', 'a4', 'a3', 'a2', 'a1']]


In [101]:
b2 = Element('a2', m5)
b3 = Element('a3', m5)

In [102]:
elem = b2
print(elem.can_add)
print(elem.can_subtract)
print(elem.can_multiply)
print(elem.can_divide)

True
False
False
False


In [103]:
b2 + b3

'a1'

In [104]:
try:
    print(b2 - b3)
except Exception as exc:
    print(exc)

M5 does not support subtraction


## Test Field

In [105]:
f5 = generate_algebra_mod_n(5)
f5.about()


** Field **
Name: F5
Instance ID: 5071215952
Description: Autogenerated Field of integers mod 5
Order: 5
Identity: 'a0'
Commutative? Yes
Cyclic?: Yes
  Generators: ['a1', 'a2', 'a3', 'a4']
Elements:
   Index   Name   Inverse  Order
      0    'a0'    'a0'       1
      1    'a1'    'a4'       5
      2    'a2'    'a3'       5
      3    'a3'    'a2'       5
      4    'a4'    'a1'       5
Cayley Table (showing indices):
[[0, 1, 2, 3, 4],
 [1, 2, 3, 4, 0],
 [2, 3, 4, 0, 1],
 [3, 4, 0, 1, 2],
 [4, 0, 1, 2, 3]]
Mult. Identity: 'a1'
Mult. Commutative? Yes
Zero Divisors: None
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4],
 [0, 2, 4, 1, 3],
 [0, 3, 1, 4, 2],
 [0, 4, 3, 2, 1]]


In [106]:
c0 = Element('a0', f5)
c1 = Element('a2', f5)
c2 = Element('a3', f5)

In [107]:
elem = c1
print(elem.can_add)
print(elem.can_subtract)
print(elem.can_multiply)
print(elem.can_divide)

True
True
True
True


In [108]:
print(c1 + c2)
print(c1 - c2)
print(c1 * c2)
print(c1 / c2)
print(c1)

a0
a4
a1
a4
a2


In [109]:
k = 4
elem = c1
print(f"{elem}^{k} = {elem**k}")

a2^4 = a1


In [110]:
try:
    print(c1**4.7)
except Exception as exc:
    print(exc)

n = 4.7, but the power must be a positive integer.


## Test Ring

In [111]:
r6 = generate_algebra_mod_n(6)
r6.about()


** Ring **
Name: R6
Instance ID: 4484231696
Description: Autogenerated Ring of integers mod 6
Order: 6
Identity: 'a0'
Commutative? Yes
Cyclic?: Yes
  Generators: ['a1', 'a5']
Elements:
   Index   Name   Inverse  Order
      0    'a0'    'a0'       1
      1    'a1'    'a5'       6
      2    'a2'    'a4'       3
      3    'a3'    'a3'       2
      4    'a4'    'a2'       3
      5    'a5'    'a1'       6
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5, 0],
 [2, 3, 4, 5, 0, 1],
 [3, 4, 5, 0, 1, 2],
 [4, 5, 0, 1, 2, 3],
 [5, 0, 1, 2, 3, 4]]
Mult. Identity: 'a1'
Mult. Commutative? Yes
Zero Divisors: ['a2', 'a3', 'a4']
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5],
 [0, 2, 4, 0, 2, 4],
 [0, 3, 0, 3, 0, 3],
 [0, 4, 2, 0, 4, 2],
 [0, 5, 4, 3, 2, 1]]


In [112]:
d0 = Element('a0', r6)
d1 = Element('a1', r6)
d2 = Element('a2', r6)
d3 = Element('a3', r6)

In [113]:
elem = d2
print(elem.can_add)
print(elem.can_subtract)
print(elem.can_multiply)
print(elem.can_divide)

True
True
True
False


In [114]:
print(d1 + d2)
print(d1 - d2)
print(d1 * d2)
#print(c1 / c2)

a3
a5
a2


In [115]:
try:
    print(d1 / d2)
except Exception as exc:
    print(exc)

R6 does not support division


In [116]:
k = 2
elem = d2
print(f"{elem}^{k} = {elem**k}")

a2^2 = a4


## Element Maps

An element map is a dictionary where the element names (strings) are the keys and their corresponding Element objects are the values.

In [143]:
s3elems = element_map(s3)
s3elems

{'(1, 2, 3)': '(1, 2, 3)',
 '(1, 3, 2)': '(1, 3, 2)',
 '(2, 1, 3)': '(2, 1, 3)',
 '(2, 3, 1)': '(2, 3, 1)',
 '(3, 1, 2)': '(3, 1, 2)',
 '(3, 2, 1)': '(3, 2, 1)'}

In [144]:
s3elems['(1, 3, 2)']

'(1, 3, 2)'

In [145]:
f5elems = element_map(f5)
f5elems

{'a0': 'a0', 'a1': 'a1', 'a2': 'a2', 'a3': 'a3', 'a4': 'a4'}

In [146]:
f5elems['a2']

'a2'